# Image Segmentation with Grounded SAM

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/grounded_sam.ipynb)

This notebook demonstrates how to use the `GroundedSAM` class to segment images using Grounded SAM. To learn more about Grounded SAM, please refer to the [Grounded SAM](https://huggingface.co/docs/transformers/model_doc/grounding-dino#grounded-sam) documentation.

## Install package

To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [ ]:
%pip install geoai-py

## Import library

In [ ]:
import geoai
from geoai.segment import GroundedSAM

## Download sample data

The sample data is from [OpenAerialMap](https://map.openaerialmap.org/#/-122.24462628364563,37.54457094085014,18/square/023010230021/60a48baa25820c0005e45906?_k=jqrisj). Credits to the provider Jason McMinn.

In [ ]:
raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/aerial.tif"

In [ ]:
raster_path = geoai.download_file(raster_url)

## Visualize sample data

In [ ]:
geoai.view_raster(raster_url)

## Image Segmentation

In [ ]:
grounded_sam = GroundedSAM(
    detector_id="IDEA-Research/grounding-dino-tiny",
    segmenter_id="facebook/sam-vit-base",
    device=None,  # Will auto-detect CUDA if available
    tile_size=1024,
    overlap=128,
    threshold=0.3,
)

In [ ]:
text_prompts = ["building", "car", "tree"]
output_file = "segmented_objects.tif"

In [ ]:
result_files = grounded_sam.segment_image(
    input_path=raster_path,
    output_path=output_file,
    text_prompts=text_prompts,
    polygon_refinement=True,
    export_boxes=True,
    export_polygons=True,
    smoothing_sigma=1.0,
    nms_threshold=0.5,
    min_polygon_area=50,
    simplify_tolerance=1.0,
)

In [ ]:
result_files

## Visualize results

In [ ]:
polygons = result_files["polygons"]
boxes = result_files["boxes"]

In [ ]:
geoai.view_vector_interactive(
    polygons, basemap=raster_url, column="label", cmap="tab20"
)

In [ ]:
geoai.view_raster(output_file, indexes=[2], basemap=raster_url, layer_name="Building")

In [ ]:
geoai.view_raster(output_file, indexes=[3], basemap=raster_url, layer_name="Car")

In [ ]:
geoai.view_raster(output_file, indexes=[4], basemap=raster_url, layer_name="Tree")